In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
df0 = pd.read_csv('nba_team_stats_shot_zones.csv')

# Display the first few rows of the DataFrame to confirm it loaded correctly
df0.head()

,Team,RA_FGM,RA_FGA,RA_FG%,NRA_FGM,NRA_FGA,NRA_FG%,MR_FGM,MR_FGA,MR_FG%,...,LC3_FG%,RC3_FGM,RC3_FGA,RC3_FG%,C3_FGM,C3_FGA,C3_FG%,AB3_FGM,AB3_FGA,AB3_FG%
0,Atlanta Hawks,19.4,29.8,65.2,8.2,20.1,40.7,2.3,5.4,41.3,...,37.6,2.6,6.1,42.0,4.3,10.7,40.1,8.6,25.8,33.3
1,Boston Celtics,15.0,21.1,70.9,5.8,13.0,44.3,2.7,6.6,40.5,...,36.9,1.9,4.6,41.3,4.0,10.3,38.9,14.4,39.8,36.2
2,Brooklyn Nets,15.0,24.2,61.9,5.8,14.8,39.1,2.8,5.9,46.6,...,34.1,2.0,5.6,35.5,3.8,10.9,34.8,10.7,28.3,37.8
3,Charlotte Hornets,15.2,24.6,61.5,5.6,14.7,38.1,3.2,8.5,36.9,...,37.8,1.6,4.5,36.1,3.5,9.4,37.0,10.8,32.1,33.7
4,Chicago Bulls,17.0,27.2,62.5,6.9,13.5,51.6,2.7,7.4,37.2,...,43.1,1.8,5.3,33.5,4.4,11.5,38.7,12.0,32.4,37.0


In [2]:
# Load the CSV files into DataFrames
df1 = pd.read_csv('nba_team_stats_shot_zones.csv')
df2 = pd.read_csv('nba_team_stats.csv')

# Perform the join on the 'Team' column
df = pd.merge(df1, df2, on='Team', how='inner')  # You can adjust the 'how' parameter as needed

# Display the first few rows of the merged DataFrame
df.head(5)

,Team,RA_FGM,RA_FGA,RA_FG%,NRA_FGM,NRA_FGA,NRA_FG%,MR_FGM,MR_FGA,MR_FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Atlanta Hawks,19.4,29.8,65.2,8.2,20.1,40.7,2.3,5.4,41.3,...,0.781,12.1,33.4,45.5,29.7,10.4,5.3,16.3,18.4,117.1
1,Boston Celtics,15.0,21.1,70.9,5.8,13.0,44.3,2.7,6.6,40.5,...,0.787,10.9,34.0,45.0,25.7,7.7,5.6,11.8,16.1,119.3
2,Brooklyn Nets,15.0,24.2,61.9,5.8,14.8,39.1,2.8,5.9,46.6,...,0.809,9.0,30.3,39.3,25.3,7.2,3.7,15.6,21.5,107.7
3,Charlotte Hornets,15.2,24.6,61.5,5.6,14.7,38.1,3.2,8.5,36.9,...,0.767,12.8,33.0,45.8,23.5,8.0,5.0,16.1,20.6,105.8
4,Chicago Bulls,17.0,27.2,62.5,6.9,13.5,51.6,2.7,7.4,37.2,...,0.798,9.2,35.9,45.1,29.6,7.5,4.7,15.1,18.3,117.9


In [3]:
# Calculate Expected Value for each shot type
# Restricted Area shots (2 points)
df['EV_RA'] = df['RA_FG%'] * 2

# Non-Restricted Area shots (2 points)
df['EV_NRA'] = df['NRA_FG%'] * 2

# Midrange shots (2 points)
df['EV_MR'] = df['MR_FG%'] * 2

# Three-point shots (3 points)
df['EV_LC3'] = df['LC3_FG%'] * 3
df['EV_RC3'] = df['RC3_FG%'] * 3
df['EV_AB3'] = df['AB3_FG%'] * 3

# Calculate total Expected Value
df['Total_EV'] = (
    df['EV_RA'] + 
    df['EV_NRA'] + 
    df['EV_MR'] + 
    df['EV_LC3'] + 
    df['EV_RC3'] + 
    df['EV_AB3']
)

# Calculate total shot attempts for each team
df['Total_FGA'] = (
    df['RA_FGA'] + 
    df['NRA_FGA'] + 
    df['MR_FGA'] + 
    df['LC3_FGA'] + 
    df['RC3_FGA'] + 
    df['AB3_FGA']
)

# Now let's calculate the optimal distribution for each shot type
shot_types = ['RA', 'NRA', 'MR', 'LC3', 'RC3', 'AB3']

for shot_type in shot_types:
    # Calculate optimal proportion based on EV
    df[f'{shot_type}_Optimal_Prop'] = (
        df[f'EV_{shot_type}'] / 
        df['Total_EV']
    )
    
    # Calculate optimal number of attempts
    df[f'{shot_type}_Optimal_FGA'] = (
        df[f'{shot_type}_Optimal_Prop'] * 
        df['Total_FGA']
    )
    
    # Calculate the difference between optimal and actual attempts
    df[f'{shot_type}_FGA_Diff'] = (
        df[f'{shot_type}_Optimal_FGA'] - 
        df[f'{shot_type}_FGA']
    )

In [4]:
# First, let's calculate Expected Value for each shot type accounting for the percentage format
shot_types = {
    'RA': 2,    # Restricted Area (2 points)
    'NRA': 2,   # Non-Restricted Area (2 points) 
    'MR': 2,    # Mid-Range (2 points)
    'LC3': 3,   # Left Corner 3 (3 points)
    'RC3': 3,   # Right Corner 3 (3 points)
    'AB3': 3    # Above Break 3 (3 points)
}

# Calculate Expected Value for each shot type, converting percentages to proper decimal form
for shot_type, points in shot_types.items():
    # Convert percentage (e.g., 64.6) to decimal (0.646) by dividing by 100
    df[f'EV_{shot_type}'] = (df[f'{shot_type}_FG%'] / 100) * points

# Calculate total Expected Value across all shot types
df['Total_EV'] = sum(
    df[f'EV_{shot_type}'] 
    for shot_type in shot_types.keys()
)

# Calculate total field goal attempts for proper shot distribution analysis
df['Total_FGA'] = sum(
    df[f'{shot_type}_FGA'] 
    for shot_type in shot_types.keys()
)

def analyze_shot_optimization(df):
    """
    Analyzes how teams should adjust their shot selection based on Expected Value.
    Takes into account that FG% is stored as whole numbers (e.g., 64.6 means 64.6%).
    
    Parameters:
    df: DataFrame containing shot data with FG% in whole number format
    
    Returns:
    DataFrame with shot optimization recommendations
    """
    analysis_results = []
    
    for _, team in df.iterrows():
        team_analysis = {'Team': team['Team']}
        
        for shot_type in shot_types.keys():
            # Convert percentage to decimal form (e.g., 64.6% → 0.646)
            fg_pct = team[f'{shot_type}_FG%'] / 100
            
            # Calculate Expected Value using the corrected percentage
            ev = fg_pct * shot_types[shot_type]
            
            # Calculate what proportion of shots should be this type based on EV
            optimal_proportion = ev / team['Total_EV']
            
            # Calculate optimal number of attempts while maintaining total shot volume
            optimal_attempts = optimal_proportion * team['Total_FGA']
            
            # Calculate the difference between optimal and actual attempts
            attempt_difference = optimal_attempts - team[f'{shot_type}_FGA']
            
            # Calculate efficiency ratio compared to league average
            avg_ev = df[f'EV_{shot_type}'].mean()
            ev_ratio = ev / avg_ev if avg_ev > 0 else 0
            
            # Store all calculated metrics for this shot type
            team_analysis.update({
                f'{shot_type}_Current_FG%': fg_pct,
                f'{shot_type}_EV': ev,
                f'{shot_type}_Current_Attempts': team[f'{shot_type}_FGA'],
                f'{shot_type}_Optimal_Attempts': optimal_attempts,
                f'{shot_type}_Attempt_Diff': attempt_difference,
                f'{shot_type}_EV_Ratio': ev_ratio
            })
        
        analysis_results.append(team_analysis)
    
    return pd.DataFrame(analysis_results)

def print_team_insights(analysis_df, team_name):
    """
    Prints detailed shot selection insights for a specific team.
    Formats percentages and numbers for clear presentation.
    """
    team_data = analysis_df[analysis_df['Team'] == team_name].iloc[0]
    
    print(f"\nShot Selection Analysis for {team_name}")
    print("=" * 50)
    
    print("\nShot Types to Increase:")
    print("-" * 30)
    
    for shot_type in shot_types.keys():
        attempt_diff = team_data[f'{shot_type}_Attempt_Diff']
        ev = team_data[f'{shot_type}_EV']
        ev_ratio = team_data[f'{shot_type}_EV_Ratio']
        current_fg = team_data[f'{shot_type}_Current_FG%']
        
        # Only show recommendations for meaningful increases (attempt_diff > 1)
        if attempt_diff > 1:
            print(f"\n{shot_type}:")
            print(f"• Current FG%: {current_fg:.1%}")  # Will show as proper percentage
            print(f"• Expected Value: {ev:.2f} points per shot")
            print(f"• Recommendation: Increase by {int(attempt_diff)} attempts")
            print(f"• Reasoning: {ev_ratio:.2f}x league average efficiency")
    
    print("\nShot Types to Decrease:")
    print("-" * 30)
    
    for shot_type in shot_types.keys():
        attempt_diff = team_data[f'{shot_type}_Attempt_Diff']
        ev = team_data[f'{shot_type}_EV']
        ev_ratio = team_data[f'{shot_type}_EV_Ratio']
        current_fg = team_data[f'{shot_type}_Current_FG%']
        
        # Only show recommendations for meaningful decreases (attempt_diff < -1)
        if attempt_diff < -1:
            print(f"\n{shot_type}:")
            print(f"• Current FG%: {current_fg:.1%}")
            print(f"• Expected Value: {ev:.2f} points per shot")
            print(f"• Recommendation: Decrease by {int(abs(attempt_diff))} attempts")
            print(f"• Reasoning: {ev_ratio:.2f}x league average efficiency")

# Example usage for the Hawks:
shot_optimization = analyze_shot_optimization(df)

In [5]:
print_team_insights(shot_optimization, "Chicago Bulls")


Shot Selection Analysis for Chicago Bulls

Shot Types to Increase:
------------------------------

NRA:
• Current FG%: 51.6%
• Expected Value: 1.03 points per shot
• Recommendation: Increase by 1 attempts
• Reasoning: 1.19x league average efficiency

MR:
• Current FG%: 37.2%
• Expected Value: 0.74 points per shot
• Recommendation: Increase by 3 attempts
• Reasoning: 0.91x league average efficiency

LC3:
• Current FG%: 43.1%
• Expected Value: 1.29 points per shot
• Recommendation: Increase by 12 attempts
• Reasoning: 1.12x league average efficiency

RC3:
• Current FG%: 33.5%
• Expected Value: 1.01 points per shot
• Recommendation: Increase by 9 attempts
• Reasoning: 0.86x league average efficiency

Shot Types to Decrease:
------------------------------

RA:
• Current FG%: 62.5%
• Expected Value: 1.25 points per shot
• Recommendation: Decrease by 9 attempts
• Reasoning: 0.94x league average efficiency

AB3:
• Current FG%: 37.0%
• Expected Value: 1.11 points per shot
• Recommendation: De

In [6]:
def analyze_team_shots(df, team_name):
    """
    Provides a detailed analysis of a team's shot distribution and recommendations
    for optimization.
    
    This function examines each shot type (Restricted Area, Non-Restricted Area,
    Mid-Range, and various three-point shots) and compares current shooting patterns
    with optimal distributions based on shooting efficiency.
    
    Parameters:
    df (DataFrame): The shot zone focused DataFrame containing shooting statistics
    team_name (str): The name of the team to analyze
    """
    # Get the specific team's data
    team_data = df[df['Team'] == team_name].iloc[0]
    
    # Calculate total field goal attempts for shot distribution analysis
    total_fga = sum(team_data[f'{shot_type}_FGA'] for shot_type in shot_types)
    
    # Calculate total Expected Value for determining optimal proportions
    total_ev = sum((team_data[f'{shot_type}_FG%'] / 100) * shot_types[shot_type] 
                   for shot_type in shot_types)
    
    print(f"\nShot Distribution Analysis for {team_name}")
    print("-" * 50)
    
    for shot_type in shot_types:
        # Calculate current shot distribution as a percentage of total attempts
        current_pct = (team_data[f'{shot_type}_FGA'] / total_fga) * 100
        
        # Calculate Expected Value for this shot type
        shot_ev = (team_data[f'{shot_type}_FG%'] / 100) * shot_types[shot_type]
        
        # Calculate optimal shot distribution based on Expected Value
        optimal_pct = (shot_ev / total_ev) * 100
        
        # Calculate the difference in number of attempts
        current_attempts = team_data[f'{shot_type}_FGA']
        optimal_attempts = (optimal_pct / 100) * total_fga
        attempt_diff = optimal_attempts - current_attempts
        
        # Print detailed analysis for this shot type
        print(f"\n{shot_type} Analysis:")
        # Convert stored percentage (e.g., 64.6) to proper percentage format
        print(f"Current FG%: {(team_data[f'{shot_type}_FG%'] / 100):.1%}")
        print(f"Expected Value: {shot_ev:.3f} points per shot")
        print(f"Current Distribution: {current_pct:.1f}% of shots")
        print(f"Optimal Distribution: {optimal_pct:.1f}% of shots")
        print(f"Suggested Change: {attempt_diff:+.0f} attempts")
        
        # Add context to help interpret the numbers
        pct_difference = optimal_pct - current_pct
        if abs(pct_difference) >= 2:  # Only show meaningful differences
            if pct_difference > 0:
                print(f"Insight: {shot_type} shots are underutilized given the "
                      f"team's {(team_data[f'{shot_type}_FG%'] / 100):.1%} "
                      f"shooting efficiency from this area.")
            else:
                print(f"Insight: {shot_type} shots might be overused considering "
                      f"the team's {(team_data[f'{shot_type}_FG%'] / 100):.1%} "
                      f"shooting efficiency from this area.")

In [7]:
analyze_team_shots(df, 'Chicago Bulls')


Shot Distribution Analysis for Chicago Bulls
--------------------------------------------------

RA Analysis:
Current FG%: 62.5%
Expected Value: 1.250 points per shot
Current Distribution: 29.6% of shots
Optimal Distribution: 19.4% of shots
Suggested Change: -9 attempts
Insight: RA shots might be overused considering the team's 62.5% shooting efficiency from this area.

NRA Analysis:
Current FG%: 51.6%
Expected Value: 1.032 points per shot
Current Distribution: 14.7% of shots
Optimal Distribution: 16.0% of shots
Suggested Change: +1 attempts

MR Analysis:
Current FG%: 37.2%
Expected Value: 0.744 points per shot
Current Distribution: 8.0% of shots
Optimal Distribution: 11.6% of shots
Suggested Change: +3 attempts
Insight: MR shots are underutilized given the team's 37.2% shooting efficiency from this area.

LC3 Analysis:
Current FG%: 43.1%
Expected Value: 1.293 points per shot
Current Distribution: 6.7% of shots
Optimal Distribution: 20.1% of shots
Suggested Change: +12 attempts
Insigh

Notes:
- Next thing to explore is how many more points will this add
- how many more makes/misses does this change
- how do free throws account in this?
- the insight portion doesn't make that much sense to me at the moment
- One more thing to note, the suggested change in shots need to equal out, for the bulls for example, it is said we need to take 27 less total shots, but only are accounting for 26 additional shots
- Goal is to create a website that users can select their favorite NBA team and look at what shots they should be taking more or less of and how it will impact their scoring averages
- I also would like to see how an increase in their scoring avearge could change their specific win total because at the end of the day, all of this is meant to focus on how we can win more games and in turn, win a championship
- I would like to create a shot chart that shows people where their team is currently shooting from, how well they are doing that, how many points are generated from a certain zone, as well as the optimized (EV) chart right next to it

In [8]:
def analyze_team_shots_with_ft(df, team_name):
    """
    Provides a detailed analysis of a team's shot distribution and recommendations,
    including free throw considerations.
    
    This function extends our previous analysis by incorporating free throws into
    the expected value calculations. It uses estimated proportions of free throws
    that come from different shot zones based on typical NBA patterns.
    
    Parameters:
    df (DataFrame): The shot zone focused DataFrame containing shooting statistics
    team_name (str): The name of the team to analyze
    """
    # Get the specific team's data
    team_data = df[df['Team'] == team_name].iloc[0]
    
    # Calculate total field goal attempts
    total_fga = sum(team_data[f'{shot_type}_FGA'] for shot_type in shot_types)
    
    # Free throw attribution weights (estimated proportions)
    ft_weights = {
        'RA': 0.70,  # 70% of FTs come from restricted area
        'NRA': 0.20,  # 20% from paint non-restricted
        'MR': 0.05,  # 5% from mid-range
        'LC3': 0.01,  # 1% from corner left threes
        'RC3': 0.01,  # 1% from corner right threes
        'AB3': 0.03  # 3% from above break threes
    }
    
    # Calculate free throw value per shot attempt for each zone
    ft_per_shot = {}
    for shot_type in shot_types:
        # Calculate attributed free throws for this zone
        zone_fta = team_data['FTA'] * ft_weights[shot_type]
        zone_attempts = team_data[f'{shot_type}_FGA']
        # Free throw points per shot attempt for this zone
        ft_per_shot[shot_type] = (zone_fta * team_data['FT%'] / 100) / zone_attempts
    
    # Calculate total Expected Value including free throws
    total_ev = sum(
        ((team_data[f'{shot_type}_FG%'] / 100) * shot_types[shot_type] +
         ft_per_shot[shot_type])
        for shot_type in shot_types
    )
    
    print(f"\nShot Distribution Analysis for {team_name} (Including Free Throws)")
    print("-" * 60)
    
    for shot_type in shot_types:
        # Calculate field goal expected value
        fg_ev = (team_data[f'{shot_type}_FG%'] / 100) * shot_types[shot_type]
        
        # Add free throw expected value
        total_shot_ev = fg_ev + ft_per_shot[shot_type]
        
        # Calculate current shot distribution
        current_pct = (team_data[f'{shot_type}_FGA'] / total_fga) * 100
        
        # Calculate optimal distribution based on total expected value
        optimal_pct = (total_shot_ev / total_ev) * 100
        
        # Calculate the difference in attempts
        current_attempts = team_data[f'{shot_type}_FGA']
        optimal_attempts = (optimal_pct / 100) * total_fga
        attempt_diff = optimal_attempts - current_attempts
        
        print(f"\n{shot_type} Analysis:")
        print(f"Current FG%: {(team_data[f'{shot_type}_FG%'] / 100):.1%}")
        print(f"Expected Points from FG: {fg_ev:.3f}")
        print(f"Expected Points from FT: {ft_per_shot[shot_type]:.3f}")
        print(f"Total Expected Value: {total_shot_ev:.3f} points per shot")
        print(f"Current Distribution: {current_pct:.1f}% of shots")
        print(f"Optimal Distribution: {optimal_pct:.1f}% of shots")
        print(f"Suggested Change: {attempt_diff:+.0f} attempts")

In [9]:
analyze_team_shots_with_ft(df, 'Chicago Bulls')


Shot Distribution Analysis for Chicago Bulls (Including Free Throws)
------------------------------------------------------------

RA Analysis:
Current FG%: 62.5%
Expected Points from FG: 1.250
Expected Points from FT: 0.004
Total Expected Value: 1.254 points per shot
Current Distribution: 29.6% of shots
Optimal Distribution: 19.5% of shots
Suggested Change: -9 attempts

NRA Analysis:
Current FG%: 51.6%
Expected Points from FG: 1.032
Expected Points from FT: 0.002
Total Expected Value: 1.034 points per shot
Current Distribution: 14.7% of shots
Optimal Distribution: 16.1% of shots
Suggested Change: +1 attempts

MR Analysis:
Current FG%: 37.2%
Expected Points from FG: 0.744
Expected Points from FT: 0.001
Total Expected Value: 0.745 points per shot
Current Distribution: 8.0% of shots
Optimal Distribution: 11.6% of shots
Suggested Change: +3 attempts

LC3 Analysis:
Current FG%: 43.1%
Expected Points from FG: 1.293
Expected Points from FT: 0.000
Total Expected Value: 1.293 points per shot


In [10]:
def analyze_scoring_impact(df, team_name):
    """
    Analyzes how a team's points per game would change if they optimized their shot distribution.
    
    This function takes into account both field goals and free throws, calculating the
    difference between current scoring and potential optimized scoring. It maintains the
    same total number of shot attempts but redistributes them according to the optimal
    proportions determined by expected value analysis.
    
    Parameters:
    df (DataFrame): The shot zone focused DataFrame containing shooting statistics
    team_name (str): The name of the team to analyze
    
    Returns:
    tuple: (current_ppg, optimized_ppg, ppg_difference)
    """
    # Get the specific team's data
    team_data = df[df['Team'] == team_name].iloc[0]
    
    # Calculate total field goal attempts
    total_fga = sum(team_data[f'{shot_type}_FGA'] for shot_type in shot_types)
    
    # Free throw attribution weights
    ft_weights = {
        'RA': 0.50,  # 60% of FTs come from restricted area
        'NRA': 0.30,  # 30% from paint non-restricted
        'MR': 0.16,  # 6% from mid-range
        'LC3': 0.01,  # 1% from corner left threes
        'RC3': 0.01,  # 1% from corner right threes
        'AB3': 0.02  # 2% from above break threes
    }
    
    # Calculate free throw value per shot attempt for each zone
    ft_per_shot = {}
    for shot_type in shot_types:
        zone_fta = team_data['FTA'] * ft_weights[shot_type]
        zone_attempts = team_data[f'{shot_type}_FGA']
        ft_per_shot[shot_type] = (zone_fta * team_data['FT%']) / zone_attempts
    
    # Calculate total Expected Value including free throws
    shot_evs = {}
    for shot_type in shot_types:
        fg_ev = (team_data[f'{shot_type}_FG%'] / 100) * shot_types[shot_type]
        shot_evs[shot_type] = fg_ev + ft_per_shot[shot_type]
    
    total_ev = sum(shot_evs.values())
    
    # Calculate current points per game
    current_points = 0
    for shot_type in shot_types:
        # Points from field goals
        fg_points = (team_data[f'{shot_type}_FGA'] * 
                    team_data[f'{shot_type}_FG%'] / 100 * 
                    shot_types[shot_type])
        
        # Points from free throws attributed to this shot type
        ft_points = team_data[f'{shot_type}_FGA'] * ft_per_shot[shot_type]
        
        current_points += fg_points + ft_points
    
    # Calculate optimized points per game
    optimized_points = 0
    for shot_type in shot_types:
        # Calculate optimal number of attempts for this shot type
        optimal_pct = shot_evs[shot_type] / total_ev
        optimal_attempts = total_fga * optimal_pct
        
        # Points from field goals with optimal distribution
        opt_fg_points = (optimal_attempts * 
                        team_data[f'{shot_type}_FG%'] / 100 * 
                        shot_types[shot_type])
        
        # Points from free throws with optimal distribution
        opt_ft_points = optimal_attempts * ft_per_shot[shot_type]
        
        optimized_points += opt_fg_points + opt_ft_points
    
    # Calculate the difference
    points_difference = optimized_points - current_points
    
    # Print detailed analysis
    print(f"\nScoring Impact Analysis for {team_name}")
    print("-" * 50)
    print(f"Current Points per Game: {current_points:.1f}")
    print(f"Optimized Points per Game: {optimized_points:.1f}")
    print(f"Potential Improvement: {points_difference:+.1f} points per game")
    
    # Show breakdown by shot type
    print("\nBreakdown by Shot Type:")
    for shot_type in shot_types:
        current_attempts = team_data[f'{shot_type}_FGA']
        optimal_pct = shot_evs[shot_type] / total_ev
        optimal_attempts = total_fga * optimal_pct
        
        print(f"\n{shot_type}:")
        print(f"Current Attempts: {current_attempts:.0f}")
        print(f"Optimal Attempts: {optimal_attempts:.0f}")
        print(f"Change in Attempts: {optimal_attempts - current_attempts:+.0f}")
    
    return current_points, optimized_points, points_difference

In [11]:
analyze_scoring_impact(df, 'Chicago Bulls')


Scoring Impact Analysis for Chicago Bulls
--------------------------------------------------
Current Points per Game: 117.8
Optimized Points per Game: 116.4
Potential Improvement: -1.5 points per game

Breakdown by Shot Type:

RA:
Current Attempts: 27
Optimal Attempts: 19
Change in Attempts: -8

NRA:
Current Attempts: 14
Optimal Attempts: 17
Change in Attempts: +3

MR:
Current Attempts: 7
Optimal Attempts: 13
Change in Attempts: +6

LC3:
Current Attempts: 6
Optimal Attempts: 16
Change in Attempts: +10

RC3:
Current Attempts: 5
Optimal Attempts: 13
Change in Attempts: +7

AB3:
Current Attempts: 32
Optimal Attempts: 14
Change in Attempts: -19


(117.8269, 116.35761789207487, -1.4692821079251246)

In [12]:
# Still needs to be tweaked as the PPG isn't corrent

# def analyze_scoring_impact_enhanced(df, team_name):
#     """
#     Analyzes team scoring with enhanced considerations for shot volume effects and free throws.
    
#     This enhanced version includes:
#     1. Volume adjustment - efficiency decreases as attempts increase beyond current levels
#     2. Direct free throw value calculation based on shot location
    
#     Parameters:
#     df (DataFrame): Shot zone focused DataFrame with team statistics
#     team_name (str): Name of the team to analyze
#     """
#     # Get team data
#     team_data = df[df['Team'] == team_name].iloc[0]
    
#     # Calculate total field goal attempts
#     total_fga = sum(team_data[f'{shot_type}_FGA'] for shot_type in shot_types)
    
#     # Volume adjustment factors - how much efficiency drops with increased volume
#     # Higher numbers mean efficiency drops more quickly with volume
#     volume_sensitivity = {
#         'RA': 0.15,      # Least sensitive - high percentage shots stay reliable
#         'NRA': 0.20,
#         'MR': 0.25,
#         'LC3': 0.30,            # More sensitive - harder to maintain efficiency
#         'RC3': 0.30,
#         'AB3': 0.20        # Moderate - volume already high
#     }
    
#     # Free throw attribution by zone (estimated from typical NBA patterns)
#     ft_rates = {
#         'RA': 0.25,      # 25% chance of free throws per attempt
#         'NRA': 0.15,
#         'MR': 0.08,
#         'LC3': 0.05,
#         'RC3': 0.05,
#         'AB3': 0.05
#     }
    
#     def calculate_adjusted_fg_percentage(original_fg_pct, current_attempts, new_attempts, sensitivity):
#         """
#         Calculates how field goal percentage changes with volume.
#         Uses a diminishing returns formula based on volume increase.
#         """
#         if new_attempts <= current_attempts:
#             return original_fg_pct
        
#         volume_ratio = new_attempts / current_attempts
#         adjustment = 1 - (volume_ratio - 1) * sensitivity
#         return original_fg_pct * adjustment
    
#     def calculate_shot_value(shot_type, attempts, original_attempts):
#         """
#         Calculates expected points per shot including both field goals and free throws,
#         with volume-adjusted shooting percentage.
#         """
#         # Get original FG%
#         original_fg_pct = team_data[f'{shot_type}_FG%'] / 100
        
#         # Calculate volume-adjusted FG%
#         adjusted_fg_pct = calculate_adjusted_fg_percentage(
#             original_fg_pct,
#             original_attempts,
#             attempts,
#             volume_sensitivity[shot_type]
#         )
        
#         # Expected points from field goals
#         fg_points = adjusted_fg_pct * shot_types[shot_type]
        
#         # Expected points from free throws
#         ft_rate = ft_rates[shot_type]
#         ft_percentage = team_data['FT%']
#         ft_points = ft_rate * 2 * ft_percentage  # Assuming 2 FTs per foul
        
#         return fg_points + ft_points
    
#     # Calculate current points
#     current_points = 0
#     for shot_type in shot_types:
#         current_attempts = team_data[f'{shot_type}_FGA']
#         shot_value = calculate_shot_value(shot_type, current_attempts, current_attempts)
#         current_points += current_attempts * shot_value
    
#     # Find optimal distribution through iterative adjustment
#     optimized_attempts = {shot_type: team_data[f'{shot_type}_FGA'] for shot_type in shot_types}
#     step_size = 0.5  # How many shots to redistribute per iteration
    
#     for _ in range(100):  # Limited iterations for convergence
#         best_gain = 0
#         best_move = None
        
#         # Try redistributing shots between each pair of zones
#         for from_type in shot_types:
#             for to_type in shot_types:
#                 if from_type != to_type and optimized_attempts[from_type] >= step_size:
#                     # Temporarily redistribute shots
#                     temp_attempts = optimized_attempts.copy()
#                     temp_attempts[from_type] -= step_size
#                     temp_attempts[to_type] += step_size
                    
#                     # Calculate new total points
#                     new_points = 0
#                     for shot_type in shot_types:
#                         shot_value = calculate_shot_value(
#                             shot_type, 
#                             temp_attempts[shot_type],
#                             team_data[f'{shot_type}_FGA']
#                         )
#                         new_points += temp_attempts[shot_type] * shot_value
                    
#                     gain = new_points - current_points
#                     if gain > best_gain:
#                         best_gain = gain
#                         best_move = (from_type, to_type)
        
#         if best_move:
#             from_type, to_type = best_move
#             optimized_attempts[from_type] -= step_size
#             optimized_attempts[to_type] += step_size
#         else:
#             break  # No more beneficial moves found
    
#     # Calculate optimized points
#     optimized_points = 0
#     for shot_type in shot_types:
#         shot_value = calculate_shot_value(
#             shot_type,
#             optimized_attempts[shot_type],
#             team_data[f'{shot_type}_FGA']
#         )
#         optimized_points += optimized_attempts[shot_type] * shot_value
    
#     # Print detailed analysis
#     print(f"\nEnhanced Scoring Analysis for {team_name}")
#     print("-" * 50)
#     print(f"Current Points per Game: {current_points:.1f}")
#     print(f"Optimized Points per Game: {optimized_points:.1f}")
#     print(f"Potential Improvement: {optimized_points - current_points:+.1f} points per game")
    
#     print("\nShot Distribution Changes:")
#     for shot_type in shot_types:
#         current = team_data[f'{shot_type}_FGA']
#         optimized = optimized_attempts[shot_type]
#         print(f"\n{shot_type}:")
#         print(f"Current Attempts: {current:.1f}")
#         print(f"Optimized Attempts: {optimized:.1f}")
#         print(f"Change: {optimized - current:+.1f}")
        
#         # Show efficiency changes
#         original_value = calculate_shot_value(shot_type, current, current)
#         new_value = calculate_shot_value(shot_type, optimized, current)
#         print(f"Points per Shot Change: {original_value:.3f} → {new_value:.3f}")
    
#     return current_points, optimized_points, optimized_points - current_points

In [14]:
# <ANTARTIFACTLINK identifier="nba-shot-analysis" type="application/vnd.ant.react" title="NBA Shot Analysis Dashboard" isClosed="true" />

# To use the analysis functions we created earlier, here's an example:

# ```python
# Example usage of our analysis functions
import pandas as pd

# Load your data
shot_data = df

# Analyze a specific team's shots
analyze_team_shots(shot_data, "Atlanta Hawks")

# Analyze scoring impact
current_ppg, optimized_ppg, points_difference = analyze_scoring_impact(shot_data, "Atlanta Hawks")

# Print the results
print(f"\nProjected Impact Analysis for Atlanta Hawks:")
print(f"Current Points per Game: {current_ppg:.1f}")
print(f"Optimized Points per Game: {optimized_ppg:.1f}")
print(f"Total Points Impact: {points_difference:.1f} points per game")

# If you want to estimate wins impact, we can add that calculation
# Using the common approximation that each point differential 
# corresponds to about 2.7 wins over a season (https://www.nbastuffer.com/analytics101/projected-winning-percentage/#:~:text=Home%20%C2%BB%20Analytics%20101%20%C2%BB%20Projected%20Winning,2.7)+41%5D/82)
projected_wins_change = (points_difference * 82) / 30  # Simplified pythag wins formula
print(f"Projected Win Impact: {projected_wins_change:.1f} additional wins")


Shot Distribution Analysis for Atlanta Hawks
--------------------------------------------------

RA Analysis:
Current FG%: 65.2%
Expected Value: 1.304 points per shot
Current Distribution: 32.5% of shots
Optimal Distribution: 20.6% of shots
Suggested Change: -11 attempts
Insight: RA shots might be overused considering the team's 65.2% shooting efficiency from this area.

NRA Analysis:
Current FG%: 40.7%
Expected Value: 0.814 points per shot
Current Distribution: 21.9% of shots
Optimal Distribution: 12.9% of shots
Suggested Change: -8 attempts
Insight: NRA shots might be overused considering the team's 40.7% shooting efficiency from this area.

MR Analysis:
Current FG%: 41.3%
Expected Value: 0.826 points per shot
Current Distribution: 5.9% of shots
Optimal Distribution: 13.0% of shots
Suggested Change: +7 attempts
Insight: MR shots are underutilized given the team's 41.3% shooting efficiency from this area.

LC3 Analysis:
Current FG%: 37.6%
Expected Value: 1.128 points per shot
Current